In [1]:
# Import all the necessary packages
#import uproot  # check out the documentation about the basics at https://uproot.readthedocs.io/en/latest/basic.html
import os
import glob
import matplotlib.pyplot as plt
import awkward as ak
import datetime as dt
import numpy as np
import vector
import itertools
vector.register_awkward()


In [2]:
DATA_DIR = "."
PATHS = list(set(glob.glob(os.path.join(DATA_DIR, '*.parquet'))))

In [3]:
inf = [i for i in PATHS if 'SMH' in i][0]
data = ak.from_parquet(inf)
data

<Array [{run: 1, ... MET_phi: 1.89}] type='105625 * {"run": int32, "luminosityBl...'>

In [4]:
# Let's create additional computed variables
start = dt.datetime.now()
# E and mu impact parameters and their significance
data['Muon_ip3d']      = np.sqrt(data.Muon_dxy*data.Muon_dxy + data.Muon_dz*data.Muon_dz)
data['Muon_sip3d']     = data.Muon_ip3d/np.sqrt(data.Muon_dxyErr*data.Muon_dxyErr + data.Muon_dzErr*data.Muon_dzErr)
data['Electron_ip3d']  = np.sqrt(data.Electron_dxy*data.Electron_dxy + data.Electron_dz*data.Electron_dz)
data['Electron_sip3d'] = data.Electron_ip3d/np.sqrt(data.Electron_dxyErr*data.Electron_dxyErr + data.Electron_dzErr*data.Electron_dzErr)
data['mus'] = vector.zip({'pt':data.Muon_pt,'eta':data.Muon_eta,'phi':data.Muon_phi,'mass':data.Muon_mass})
stop = dt.datetime.now()
print(stop-start)

0:00:00.081744


In [5]:
# Let's filter by quality

def createFilter(data,minMu,minEl):
    mu_isolation_cut = ak.sum(abs(data.Muon_pfRelIso04_all) < 0.4, axis=1) == minMu
    muon_pt_cut = ak.sum(data.Muon_pt > 5, axis=1) ==minMu
    muon_eta_cut = ak.sum(abs(data.Muon_eta) < 2.4, axis=1) == minMu
    muon_sip_cut = ak.sum(abs(data.Muon_sip3d) < 4, axis=1) == minMu
    muon_dxy_cut = ak.sum(abs(data.Muon_dxy) < 0.5, axis=1) == minMu
    muon_dz_cut = ak.sum(abs(data.Muon_dz) < 1.0, axis=1) == minMu
    mu_cuts = mu_isolation_cut * muon_pt_cut * muon_eta_cut * muon_sip_cut * muon_dxy_cut * muon_dz_cut

    el_isolation_cut = ak.sum(abs(data.Electron_pfRelIso03_all) < 0.4, axis=1) == minEl
    el_pt_cut = ak.sum(data.Electron_pt > 7, axis=1) ==minEl
    el_eta_cut = ak.sum(abs(data.Electron_eta) < 2.5, axis=1) == minEl
    el_sip_cut = ak.sum(abs(data.Electron_sip3d) < 4, axis=1) == minEl
    el_dxy_cut = ak.sum(abs(data.Electron_dxy) < 0.5, axis=1) == minEl
    el_dz_cut = ak.sum(abs(data.Electron_dz) < 1.0, axis=1) == minEl
    el_cuts = el_isolation_cut * el_pt_cut * el_eta_cut * el_sip_cut * el_dxy_cut * el_dz_cut

    return mu_cuts * el_cuts

start=dt.datetime.now()

fourMu    = createFilter(data,4,0)
twoMuTwoE = createFilter(data,2,2)
fourEl    = createFilter(data,0,4)

stop=dt.datetime.now()
print(stop-start)

0:00:00.346779


In [6]:
# Let's make arrays for the three final states
start=dt.datetime.now()

hzz4mu=data[fourMu].to_list()
hzz2e2mu=data[twoMuTwoE].to_list()
hzz4e=data[fourEl].to_list()
stop=dt.datetime.now()


print(stop-start)

0:00:14.673632


In [66]:
# Sample sizes:
print("Higgs to ZZ to 4 muons sample size: %i" % len(hzz4mu))
print("Higgs to ZZ to 2 electrons and 2 muons sample size: %i" % len(hzz2e2mu))
print("Higgs to ZZ to 4 electrons sample size: %i" % len(hzz4e))

Higgs to ZZ to 4 muons sample size: 8723
Higgs to ZZ to 2 electrons and 2 muons sample size: 12730
Higgs to ZZ to 4 electrons sample size: 5706


In [8]:
itertools.combinations(hzz4mu[0]['Muon_pt'], 2)

In [9]:
x = itertools.combinations(range(4),2)
for i in x:
    print(i,'\t', i[0], '\t', i[1])


(0, 1) 	 0 	 1
(0, 2) 	 0 	 2
(0, 3) 	 0 	 3
(1, 2) 	 1 	 2
(1, 3) 	 1 	 3
(2, 3) 	 2 	 3


In [41]:
start=dt.datetime.now()
zmass = []
count = 0
for e in hzz4mu:
    nMuon = e['nMuon'] # How many compnents in muon vector
    mus = []
    for i in range(nMuon):
        mus.append(vector.obj(pt=e['Muon_pt'][i],eta=e['Muon_eta'][i],
                              phi=e['Muon_phi'][i],mass=e['Muon_mass'][i])) # list of vectors
        
    combos = itertools.combinations(range(nMuon),2) # find combinations like (0, 1), (0, 2),...,(2, 3)
    for j in combos:                                # in total there are 6 possible combinations bc of 4 comp
        m1 = mus[j[0]] # 1st mass of the combination
        m2 = mus[j[1]] # 2nd mass of the combination
        z = m1 + m2              # sum of the muon masses 
        ch1 = e['Muon_charge'][j[0]] # 1st charge of the combination
        ch2 = e['Muon_charge'][j[1]] # 2nd charge of the combination
        if not (ch1 + ch2): # '+/-' and '+/-' not possible, only '+' and '-'
            count = count + 1
            zmass.append(z.mass) # find leptons with different charge combinations 
                                 # and write into a list named 'zmass'

stop=dt.datetime.now()
print(count)      # how many components are in zmass
print(zmass[0:5]) # print first 5 sums of masses for show
print(stop-start)

[7.725514161258727, 55.01387254537647, 21.10976537457914, 93.66340214757945, 58.027073599155536]
34895
0:00:08.520506


In [65]:
#zmass

In [63]:
# Function to find element
# closest to given target.
# LINK: https://stackoverflow.com/questions/2566412/find-nearest-value-in-numpy-array
# Returns element closest to target in array
def find_nearest(array, target):
    array = np.asarray(array)
    idx = (np.abs(array - target)).argmin()
    return array[idx]

In [67]:
find_nearest(zmass, 93)

93.00144433793005